In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from catboost import CatBoostRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold as skf
import matplotlib.pyplot as plt

In [2]:
files = os.listdir('final_data')
test = pd.read_csv('final_test\\cleaned.csv')

In [3]:
test.drop(columns = ['wind_dir'], inplace = True)

In [4]:
train = pd.read_csv('final_data\\' + files[0])
train['location'] = [files[0][0].upper()]* len(train)

In [5]:
for i in tqdm(range(1, len(files))):
    data = pd.read_csv('final_data\\' + files[i])
    data['location'] = [files[i][0].upper()]* len(data)
    train = pd.concat([train,data],sort=False).reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.88it/s]


In [6]:
del data
features=["temp","precip","rel_humidity","wind_dir_x","wind_dir_y","wind_spd","atmos_press"]
for feature in tqdm(features) : 
    train[feature]=train[feature].apply(lambda x: [ float(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ float(X)  for X in x.replace("nan"," ").split(",")]) 

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.02s/it]


In [7]:
# def wind(wind_spd, wind_dir):
#     x = np.sin(np.asarray(wind_dir) * 2 * np.pi/ 360) * np.asarray(wind_spd)
#     y = np.cos(np.asarray(wind_dir) * 2 * np.pi/ 360) * np.asarray(wind_spd)
#     return x, y
# x = []
# y = []
# for i in tqdm(range(len(test))):
#     wind_spd = np.asarray(test.iloc[i].wind_spd)
#     wind_dir = np.asarray(test.iloc[i].wind_dir)
#     X = wind(wind_spd, wind_dir)
#     x.append(X[0])
#     y.append(X[1])
# test['wind_x'] = x
# test['wind_y'] = y

In [8]:
train = pd.concat([train,test],sort=False).reset_index(drop=True)

In [9]:
# x = []
# y = []
# for i in tqdm(range(len(train))):
#     wind_spd = np.asarray(train.iloc[i].wind_spd)
#     wind_dir_x = np.asarray(train.iloc[i].wind_dir_x)
#     wind_dir_y = np.asarray(train.iloc[i].wind_dir_y)
#     x.append(wind_dir_x * wind_spd)
#     y.append(wind_dir_y * wind_spd)
# train['wind_x'] = x
# train['wind_y'] = y



In [10]:
# train.drop(columns = ['wind_dir_x', 'wind_dir_y', 'wind_spd'], inplace = True)

In [11]:
# train = pd.concat([train,test],sort=False).reset_index(drop=True)

In [12]:
train.head()

,temp,atmos_press,precip,wind_spd,wind_dir_x,wind_dir_y,rel_humidity,target,location,ID
0,"[22.126603934041547, 21.217763029093668, 20.54...","[88.59485207367172, 88.65705733877957, 88.7216...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.92509967290915, 0.704266903389754, 0.605630...","[0.2376490142083889, 0.2376490142083889, 0.237...","[0.9713510930893016, 0.9713510930893016, 0.971...","[0.7522369831437413, 0.7919130411237965, 0.825...",39.640833,A,NaN
1,"[21.217763029093668, 20.546494676024608, 20.09...","[88.65705733877957, 88.7216007618365, 88.77434...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.704266903389754, 0.6056301047996498, 0.5884...","[0.2376490142083889, 0.2376490142083889, 0.237...","[0.9713510930893016, 0.9713510930893016, 0.971...","[0.7919130411237965, 0.8253744374127658, 0.846...",71.478667,A,NaN
2,"[20.546494676024608, 20.099446346767596, 19.76...","[88.7216007618365, 88.77434829365376, 88.80083...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6056301047996498, 0.5884308409586871, 0.586...","[0.2376490142083889, 0.2376490142083889, 0.237...","[0.9713510930893016, 0.9713510930893016, 0.971...","[0.8253744374127658, 0.8468925128562416, 0.859...",39.516000,A,NaN
3,"[20.099446346767596, 19.761651951067364, 19.43...","[88.77434829365376, 88.80083315907484, 88.7929...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5884308409586871, 0.5864472250235723, 0.565...","[0.2376490142083889, 0.2376490142083889, 0.237...","[0.9713510930893016, 0.9713510930893016, 0.971...","[0.8468925128562416, 0.859290609200586, 0.8703...",33.636875,A,NaN
4,"[19.761651951067364, 19.439852165723998, 19.15...","[88.80083315907484, 88.79296733724759, 88.7554...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5864472250235723, 0.5657670189840984, 0.538...","[0.2376490142083889, 0.2376490142083889, 0.237...","[0.9713510930893016, 0.9713510930893016, 0.971...","[0.859290609200586, 0.8703615859582586, 0.8842...",39.375682,A,NaN


In [13]:
def day_time(x):
    if(x < 0):
        time = 24 - x % 24
    else: time = x % 24
    return time

def feels_like(temp, rel_hum):
    c1 = -8.78469475556
    c2 = 1.61139411
    c3 = 2.33854883889
    c4 = -.14611605
    c5 = -.012308094
    c6 = -.0164248277778
    c7 = .002211732
    c8 = 0.00072546
    c9 = -.000003582
    hi = c1 + c2 * temp + c3 * rel_hum + c4 * temp * rel_hum + c5 * np.square(temp) + c6 * np.square(rel_hum) + c7 * np.square(temp) * rel_hum + c8 * np.square(rel_hum) * temp + c9 * np.square(temp) * np.square(rel_hum)
    return hi

def wind_chill(temp, wind_spd):
    return 35.74 + 0.6215 * temp - 35.75 * (wind_spd ** 0.16) +0.4275 * temp * (wind_spd ** 0.16)


time = []
# feels = []
for i in tqdm(range(len(train))):
    temp = np.asarray(train.iloc[i].temp)
#     rel_hum = np.asarray(train.iloc[i].rel_humidity) * 100
#     feels.append(feels_like(temp, rel_hum))
    max_time = 15
    ind = np.where(temp == np.nanmax(temp))[0][0]
    for j in range(ind + 1, 121):
        max_time += 1
    time.append(day_time(max_time))
#     wind_chill_temp.append(wind_chill(temp, wind_spd))

train['time'] = time
# train['feels'] = feels

100%|██████████████████████████████████████████████████████████████████████████| 20340/20340 [00:04<00:00, 4678.64it/s]


In [14]:
for x in tqdm(range(121)):
    train["newtemp"+ str(x)] = train.temp.str[x]
    train["newprecip"+ str(x)] = train.precip.str[x]
    train["newrel_humidity"+ str(x)] = train.rel_humidity.str[x]
    train["newwind_x"+ str(x)] = train.wind_dir_x.str[x]
    train["newwind_y"+ str(x)] = train.wind_dir_y.str[x]
#     train["newwind_dir_y"+ str(x)] = train.wind_dir_y.str[x]
    train["windspeed"+ str(x)] = train.wind_spd.str[x]
    train["atmospherepressure"+ str(x)] = train.atmos_press.str[x]
#     train["Feels"+ str(x)] = train.feels.str[x]
    

100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:15<00:00,  8.04it/s]


In [15]:
feat = ['temp', 'atmos_press', 'precip', 'rel_humidity', 'wind_spd', 'wind_dir_x', 'wind_dir_y']#, 'wind_x', 'wind_y']#, 'feels']
train.drop(feat,1,inplace=True)

In [16]:
train.head()

,target,location,ID,time,newtemp0,newprecip0,newrel_humidity0,newwind_x0,newwind_y0,windspeed0,...,newwind_y119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_x120,newwind_y120,windspeed120,atmospherepressure120
0,39.640833,A,NaN,17,22.126604,0.0,0.752237,0.237649,0.971351,0.925100,...,-0.806116,1.555000,88.334167,24.358333,0.000,0.679417,-0.945477,-0.325688,1.035833,88.400000
1,71.478667,A,NaN,18,21.217763,0.0,0.791913,0.237649,0.971351,0.704267,...,-0.325688,1.035833,88.400000,23.091667,0.000,0.732500,-0.453588,0.891212,0.801667,88.523333
2,39.516000,A,NaN,19,20.546495,0.0,0.825374,0.237649,0.971351,0.605630,...,0.891212,0.801667,88.523333,21.066667,0.000,0.785667,0.771533,-0.636189,1.611667,88.650000
3,33.636875,A,NaN,20,20.099446,0.0,0.846893,0.237649,0.971351,0.588431,...,-0.636189,1.611667,88.650000,20.416667,0.017,0.797500,0.715430,0.698684,0.836667,88.681667
4,39.375682,A,NaN,21,19.761652,0.0,0.859291,0.237649,0.971351,0.586447,...,0.698684,0.836667,88.681667,19.883333,0.000,0.845333,0.980079,0.198608,0.642500,88.720833


In [ ]:
# X=train.drop(['target'],axis =1)
# y = train.target
# errcb2=[]

# fold=skf(n_splits=20, random_state=0)
# i=1
# y_pred_totcb2=[]
# for train_index, test_index in fold.split(X,X.location):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     print(np.unique(X_train.location, return_counts = True))
#     print()
#     print(np.unique(X_test.location, return_counts = True))
#     print()

In [17]:
cleanup_nums = {"location":     {"A": 1, "B": 2, 'C': 3, 'D': 4, 'E': 5}}
train.replace(cleanup_nums, inplace=True)

In [18]:
test=train[train.target.isna()].reset_index(drop=True)
train=train[train.target.notnull()].reset_index(drop=True)

In [19]:
test

,target,location,ID,time,newtemp0,newprecip0,newrel_humidity0,newwind_x0,newwind_y0,windspeed0,...,newwind_y119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_x120,newwind_y120,windspeed120,atmospherepressure120
0,NaN,3,ID_test_0,20,23.966667,0.000,0.689833,-0.938883,0.344237,0.633333,...,-0.975964,1.289167,87.580000,22.766667,0.000,0.725250,0.281213,-0.959646,0.774167,87.625833
1,NaN,3,ID_test_1,7,18.991667,0.000,0.999750,-0.904566,0.426333,0.640833,...,-0.994772,0.665833,87.920000,20.025000,0.000,0.886083,0.930041,-0.367456,0.311667,87.998333
2,NaN,4,ID_test_10,15,25.116667,0.000,0.760167,0.974611,0.223903,2.013333,...,-0.165091,2.285833,90.409167,20.608333,1.327,0.910167,0.929732,0.368238,1.395000,90.464167
3,NaN,3,ID_test_100,9,18.716667,0.000,0.951417,-0.757555,0.652772,0.745000,...,0.862905,0.705833,87.844167,21.483333,0.000,0.861333,-0.489714,0.871883,1.026667,87.894167
4,NaN,4,ID_test_1000,5,17.633333,0.000,0.900000,-0.428850,0.903376,0.620833,...,0.975744,0.373333,90.561667,19.833333,0.000,0.851667,0.470860,0.882208,0.325000,90.584167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,NaN,3,ID_test_995,12,25.925000,0.000,0.645417,-0.502538,0.864555,0.749167,...,0.972992,0.558333,88.085833,22.058333,0.000,0.815833,-0.266279,-0.963896,0.480000,88.085000
5031,NaN,4,ID_test_996,23,22.166667,0.000,0.822750,-0.932795,0.360407,0.980833,...,-0.870761,0.879167,90.631667,19.350000,0.000,0.919167,-0.010051,-0.999949,0.567500,90.611667
5032,NaN,5,ID_test_997,6,17.358333,0.034,0.875833,0.324333,0.945943,1.439167,...,0.996148,1.563333,88.290833,20.583333,0.000,0.775000,-0.218797,0.975770,1.727500,88.350833
5033,NaN,1,ID_test_998,10,24.166667,0.000,0.831667,-0.731011,0.682366,1.706667,...,-0.428699,0.896667,88.495000,25.941667,0.000,0.504167,-0.996927,0.078340,1.110000,88.438333


In [20]:
X=train.drop(['target', 'ID'],axis =1)
y = train.target
errcb2=[]

fold=skf(n_splits=20, random_state=0, shuffle=True)
i=1
y_pred_totcb2=[]
for train_index, test_index in fold.split(X,X.location):
    print('#'*5 + str(i) + '#'*5)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators=500,
                            eval_metric='RMSE',learning_rate=0.1,
                            random_seed= 1234, use_best_model=True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(['ID', 'target'], axis = 1))
    y_pred_totcb2.append(p2)
    i+=1

#####1#####
0:	learn: 35.3767755	test: 35.3767755	test1: 34.8875088	best: 34.8875088 (0)	total: 244ms	remaining: 2m 1s
100:	learn: 22.3013822	test: 22.3013822	test1: 23.7747283	best: 23.7747283 (100)	total: 12.2s	remaining: 48.2s
200:	learn: 19.6720007	test: 19.6720007	test1: 23.0200379	best: 23.0059355 (199)	total: 23.9s	remaining: 35.5s
300:	learn: 17.6866634	test: 17.6866634	test1: 22.5085275	best: 22.5085275 (300)	total: 35.6s	remaining: 23.5s
400:	learn: 16.1274449	test: 16.1274449	test1: 22.2612117	best: 22.2612117 (400)	total: 47.5s	remaining: 11.7s
499:	learn: 14.7003415	test: 14.7003415	test1: 21.9672378	best: 21.9576237 (497)	total: 59.1s	remaining: 0us

bestTest = 21.95762372
bestIteration = 497

Shrink model to first 498 iterations.
err:  21.957623575341778
#####2#####
0:	learn: 35.3093608	test: 35.3093608	test1: 35.6307186	best: 35.6307186 (0)	total: 175ms	remaining: 1m 27s
100:	learn: 22.2326466	test: 22.2326466	test1: 24.5747395	best: 24.5747395 (100)	total: 12.3s	remain

#####12#####
0:	learn: 35.1790049	test: 35.1790049	test1: 39.3311692	best: 39.3311692 (0)	total: 135ms	remaining: 1m 7s
100:	learn: 22.3959238	test: 22.3959238	test1: 24.7636990	best: 24.7636990 (100)	total: 12.3s	remaining: 48.4s
200:	learn: 19.6889415	test: 19.6889415	test1: 24.0158495	best: 24.0158495 (200)	total: 24s	remaining: 35.8s
300:	learn: 17.6588627	test: 17.6588627	test1: 23.7433798	best: 23.7433798 (300)	total: 36s	remaining: 23.8s
400:	learn: 16.0102278	test: 16.0102278	test1: 23.5206763	best: 23.5206763 (400)	total: 47.9s	remaining: 11.8s
499:	learn: 14.6776044	test: 14.6776044	test1: 23.2622246	best: 23.2483307 (493)	total: 59.5s	remaining: 0us

bestTest = 23.24833069
bestIteration = 493

Shrink model to first 494 iterations.
err:  23.24833074911815
#####13#####
0:	learn: 35.3882610	test: 35.3882610	test1: 34.6145903	best: 34.6145903 (0)	total: 156ms	remaining: 1m 17s
100:	learn: 22.2068812	test: 22.2068812	test1: 24.1537894	best: 24.1537894 (100)	total: 12.4s	remaining

In [21]:
np.mean(errcb2)

22.7041551020262

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(m2.get_feature_importance(prettified = True).iloc[:20]['Feature Id'], m2.get_feature_importance(prettified = True).iloc[:20]['Importances'])
plt.xticks(rotation = 90)

In [22]:
d = {'ID': test['ID'], 'target': np.mean(y_pred_totcb2, 0)}
sub2 = pd.DataFrame(data=d)
sub2 = sub2[['ID', 'target']]
sub2.to_csv("sub_5.csv",index  = False)


In [ ]:
sub2

In [ ]:
plt.scatter(sub2.index, sub2.target, alpha = 0.1)

In [ ]:
id_final = []
for i in range(len(sub2)):
    id_final.append(int(sub2.iloc[i].ID.split('_')[2]))
sub2['ID_final'] = id_final

In [ ]:
sub2.sort_values('ID_final', inplace = True)

In [ ]:
sub2.drop('ID_final', axis = 1, inplace = True)

In [ ]:
id_final = []
for i in range(len(sub2)):
    id_final.append('ID_test_' + str(sub2.iloc[i].ID.split('_')[2]))
sub2['ID'] = id_final

In [ ]:

sub2.to_csv("sub_5_pca.csv",index  = False)
sub2

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X=train.drop(['target', 'ID', 'location', 'time'],axis =1)
scaler.fit(X)

In [ ]:
X = scaler.transform(X)

In [ ]:
pca = PCA()

In [ ]:
pca.fit(X)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
X = pca.transform(X)[:,:300]

In [ ]:
X.shape

In [ ]:
test.columns

In [ ]:
test_y = test.drop(['target', 'ID', 'location', 'time'],axis =1)

In [ ]:
test_y

In [ ]:
test_y = scaler.transform(test_y)

In [ ]:
test_y = pca.transform(test_y)[:,:300]

In [ ]:
test_y.shape

In [ ]:
t = pd.DataFrame(test_y)

In [ ]:
t.head()

In [ ]:
train_pca = pd.DataFrame(X)
test_pca = pd.DataFrame(test_y)

In [ ]:
train_pca['location'] = train['location']
train_pca['time'] = train['time']
train_pca['target'] = train['target']

In [ ]:
test_pca['location'] = test['location']
test_pca['time'] = test['time']
test_pca['ID'] = test['ID']